# Kafkaの起動を行います

/home/pyspark/kafka/bin/kafka-topics.sh \
    --create --topic pyspark-topic1 \
    --replication-factor 1 \
    --partitions 3 \
    --bootstrap-server kafka:9092 

In [28]:
!pip install kafka-python

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.


In [1]:
# コンソールで設定したSparkとNoteBookを接続します(動かす前に毎度実行する必要があります)
import findspark
findspark.init("/home/pyspark/spark")

In [29]:
from pyspark.streaming.kafka import KafkaUtils

ModuleNotFoundError: No module named 'pyspark.streaming.kafka'

In [2]:
#pysparkに必要なライブラリを読み込む
from pyspark import SparkConf
from pyspark import SparkContext
from pyspark.sql import SparkSession

#spark sessionの作成
# spark.ui.enabled trueとするとSparkのGUI画面を確認することができます
# spark.eventLog.enabled true　とすると　GUIで実行ログを確認することができます
# GUIなどの確認は最後のセクションで説明を行います。
spark = SparkSession.builder \
    .appName("chapter1") \
    .config("hive.exec.dynamic.partition", "true") \
    .config("hive.exec.dynamic.partition.mode", "nonstrict") \
    .config("spark.sql.session.timeZone", "JST") \
    .config("spark.ui.enabled","true") \
    .config("spark.eventLog.enabled","true") \
    .config("spark.jars.packages", "org.apache.spark:spark-streaming_2.13:3.2.0,org.apache.spark:spark-sql-kafka-0-10_2.12:3.2.0,org.apache.spark:spark-avro_2.12:3.2.0") \
    .enableHiveSupport() \
    .getOrCreate()

# パッケージを複数渡したい時は「,」で繋いで渡します。

:: loading settings :: url = jar:file:/home/pyspark/spark-3.2.0-bin-hadoop3.2/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/pyspark/.ivy2/cache
The jars for the packages stored in: /home/pyspark/.ivy2/jars
org.apache.spark#spark-streaming_2.13 added as a dependency
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
org.apache.spark#spark-avro_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-f8b0393a-64f6-48df-9994-5cd7af29e62e;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.2.0 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.2.0 in central
	found org.apache.kafka#kafka-clients;2.8.0 in central
	found org.lz4#lz4-java;1.7.1 in central
	found org.xerial.snappy#snappy-java;1.1.8.4 in central
	found org.slf4j#slf4j-api;1.7.30 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.1 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.hadoop#hadoop-client-api;3.3.1 in central
	found org.apache.htrace#htrace-core4;4.1.0-incubating in ce

# WebUIでライブラリが保存されているか確認してみましょう

100000

In [67]:
# PySparkをKafkaと接続します

# Subscribe to 1 topic
df = spark \
  .readStream \
  .format("kafka") \
  .option("kafka.bootstrap.servers", "kafka:9092") \
  .option("subscribe", "pyspark-topic1") \
  .load()
df.selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)")

DataFrame[key: string, value: string]

In [72]:
df.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



Thu Dec 02 12:54:00 UTC 2021 WARN: Establishing SSL connection without server's identity verification is not recommended. According to MySQL 5.5.45+, 5.6.26+ and 5.7.6+ requirements SSL connection must be established by default if explicit option isn't set. For compliance with existing applications not using SSL the verifyServerCertificate property is set to 'false'. You need either to explicitly disable SSL by setting useSSL=false, or set useSSL=true and provide truststore for server certificate verification.
Thu Dec 02 12:54:00 UTC 2021 WARN: Establishing SSL connection without server's identity verification is not recommended. According to MySQL 5.5.45+, 5.6.26+ and 5.7.6+ requirements SSL connection must be established by default if explicit option isn't set. For compliance with existing applications not using SSL the verifyServerCertificate property is set to 'false'. You need either to explicitly disable SSL by setting useSSL=false, or set useSSL=true and provide truststore for s

In [43]:
ds = df \
  .selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)") \
  .writeStream \
  .format("kafka") \
  .option("kafka.bootstrap.servers", "kafka:9092") \
  .option("topic", "pyspark-topic1") \
  .option("checkpointLocation","/tmp/kafka") \
  .queryName("aggregates") \
  .start()

# formatは色々変更できる

21/12/02 10:52:53 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
21/12/02 10:52:53 WARN StreamingQueryManager: Stopping existing streaming query [id=7aa56c59-0860-455e-89c4-0488457a952f, runId=b1572470-126b-4336-a2c6-6390dc692094], as a new run is being started.


In [79]:
ds2 = df \
  .selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)") \
  .writeStream \
  .format("memory") \
  .queryName("aggregates4") \
  .start()

21/12/02 13:04:50 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-77a67d7d-9cba-4e22-bb4f-3d5f43fdc8c4. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
21/12/02 13:04:50 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


In [82]:
ds2.stop()

In [75]:
ds3 = df \
  .selectExpr("'key' as key", "'1' as value") \
  .writeStream \
  .format("console") \
  .start()

21/12/02 12:55:01 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-67fd9784-8429-4a7c-b1df-a2db00eed9db. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
21/12/02 12:55:01 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


-------------------------------------------
Batch: 0
-------------------------------------------
+---+-----+
|key|value|
+---+-----+
+---+-----+

-------------------------------------------
Batch: 1
-------------------------------------------
+---+-----+
|key|value|
+---+-----+
|key|    1|
|key|    1|
+---+-----+



In [81]:
spark.sql("select * from aggregates4 ").show()

+---+-----+
|key|value|
+---+-----+
|  1|    2|
+---+-----+



In [83]:
ds3.stop()

In [58]:
ds2.stop()

In [36]:
df.selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)").show()

AnalysisException: Queries with streaming sources must be executed with writeStream.start();
kafka

Thu Dec 02 10:46:53 UTC 2021 WARN: Establishing SSL connection without server's identity verification is not recommended. According to MySQL 5.5.45+, 5.6.26+ and 5.7.6+ requirements SSL connection must be established by default if explicit option isn't set. For compliance with existing applications not using SSL the verifyServerCertificate property is set to 'false'. You need either to explicitly disable SSL by setting useSSL=false, or set useSSL=true and provide truststore for server certificate verification.
Thu Dec 02 10:46:53 UTC 2021 WARN: Establishing SSL connection without server's identity verification is not recommended. According to MySQL 5.5.45+, 5.6.26+ and 5.7.6+ requirements SSL connection must be established by default if explicit option isn't set. For compliance with existing applications not using SSL the verifyServerCertificate property is set to 'false'. You need either to explicitly disable SSL by setting useSSL=false, or set useSSL=true and provide truststore for s

In [ ]:
spark.sql("select * from activity_counts").show()

In [84]:
ds.stop()

In [ ]:
# kafkaのダウンロード
wget https://dlcdn.apache.org/kafka/3.0.0/kafka-3.0.0-src.tgz

In [ ]:
bin/kafka-topics.sh \
    --delete --topic quickstart-events \
    --bootstrap-server localhost:9092 

In [ ]:
# 繋ぐコマンド
# ちょこちょこバグがあって、接続できない時があるので、今回の講座で使っているバージョンをおさらいします。

# Spark 3.2
# Java11(openjdk version "11.0.12)
# Kafkka(3.0.0)
# 
pyspark --packages org.apache.spark:spark-sql-kafka-0-10_2.12:3.0.3

In [13]:
df_f=spark.sql("select 'key' as key ,'1' as value")
df_f.writeStream

In [8]:
from pyspark.sql.avro.functions import from_avro, to_avro

jsonFormatSchema = open("/home/pyspark/pyspark_streaming/schema/schema.avc", "r").read()
from confluent_kafka import Producer



In [ ]:
spark.stop()
spark.sparkContext.stop()